In [15]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append("..")

from datasets.img_dataset import FoodImgs
from models.cnn import SnapSnack

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [16]:
import pytorch_lightning as pl
from torch.utils.data import DataLoader, random_split
import math


dataset = FoodImgs()

s = len(dataset)
print(f"{s} imgs")
train_size = int(math.ceil(s * 0.8))
test_size = s - train_size

train_set, test_set = random_split(dataset, [train_size, test_size])


9100 imgs


In [22]:
from pytorch_lightning.loggers import WandbLogger
import wandb
# wandb.init(project="snapnsnack")
wandb_logger = WandbLogger(project='snapnsnack')
model = SnapSnack()
trainer = pl.Trainer(logger=wandb_logger)
trainer.fit(model, DataLoader(train_set), DataLoader(test_set))

GPU available: False, used: False
TPU available: False, using: 0 TPU cores



  | Name     | Type   | Params
------------------------------------
0 | backbone | ResNet | 11 M  


wandb: WARNING Step must only increase in log calls.  Step 49 < 50; dropping {'train_loss': 216.3025665283203, 'epoch': 0}.
wandb: WARNING Step must only increase in log calls.  Step 99 < 100; dropping {'train_loss': 41.74549102783203, 'epoch': 0}.
wandb: WARNING Step must only increase in log calls.  Step 149 < 150; dropping {'train_loss': 472.64166259765625, 'epoch': 0}.
wandb: WARNING Step must only increase in log calls.  Step 199 < 200; dropping {'train_loss': 27.966970443725586, 'epoch': 0}.
wandb: WARNING Step must only increase in log calls.  Step 249 < 250; dropping {'train_loss': 191.43173217773438, 'epoch': 0}.
wandb: WARNING Step must only increase in log calls.  Step 299 < 300; dropping {'train_loss': 140.13015747070312, 'epoch': 0}.
Exception ignored in: <bound method tqdm.__del__ of Epoch 0:   0%|<bar/>| 0/7280 [04:14<?, ?it/s]>
Traceback (most recent call last):
  File "/opt/anaconda3/envs/py36/lib/python3.6/site-packages/tqdm/std.py", line 1122, in __del__
    self.clo

RuntimeError: The size of tensor a (4) must match the size of tensor b (3) at non-singleton dimension 0

In [29]:
import torch
import pickle as pkl
import pandas as pd
epi = pd.read_csv("../data/snapnsnackdb/epi_r.csv")

epi.head()

,title,rating,calories,protein,fat,sodium,#cakeweek,#wasteless,22-minute meals,3-ingredient recipes,...,yellow squash,yogurt,yonkers,yuca,zucchini,cookbooks,leftovers,snack,snack week,turkey
0,"Lentil, Apple, and Turkey Wrap",2.500,426.0,30.0,7.0,559.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,Boudin Blanc Terrine with Red Onion Confit,4.375,403.0,18.0,23.0,1439.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Potato and Fennel Soup Hodge,3.750,165.0,6.0,7.0,165.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Mahi-Mahi in Tomato Olive Sauce,5.000,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Spinach Noodle Casserole,3.125,547.0,20.0,32.0,452.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [30]:
img_4 = "/Users/davinan/Dropbox/github/Snap-N-Snack/data/snapnsnackdb/simple_images/Spinach+Noodle+Casserole+/Spinach+Noodle+Casserole+_1.jpeg"

In [35]:
import matplotlib.pyplot as plt
im = plt.imread(img_4)

In [43]:
from torchvision import transforms as T
from PIL import Image

transform = T.Compose(
            [
                T.ToTensor(),
                T.Normalize(
                    mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]
                ),
                T.Resize(256),
                T.CenterCrop(224),
            ]
        )

model.eval()
oi = transform(Image.fromarray(im)).unsqueeze(0)
preds = model(oi)

In [44]:
preds

tensor([[  3.7612, 327.7388,  11.6335,  22.7230, 274.9613]],
       grad_fn=<AddmmBackward>)